# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 80 new papers today
          33 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/32 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.11942


extracting tarball to tmp_2301.11942...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.11942/tng.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'tng.bbl' from 'tmp_2301.11942/tng.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'tng.bbl' from 'tmp_2301.11942/TNG project_accepted/tng.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.12918/mr3.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'top-apj' from 'tmp_2301.12918/top-apj.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2301.12919...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.12976


extracting tarball to tmp_2301.12976...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.11942-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.11942) | **The Origin of Stars in the Inner 500 Parsecs in TNG50 Galaxies**  |
|| Alina Boecker, et al. -- incl., <mark>Nadine Neumayer</mark>, <mark>Annalisa Pillepich</mark>, <mark>Neige Frankel</mark>, <mark>Lars Hernquist</mark> |
|*Appeared on*| *2023-01-31*|
|*Comments*| *24 pages, 13 Figures, published in MNRAS*|
|**Abstract**| We investigate the origin of stars in the innermost $500\,\mathrm{pc}$ of galaxies spanning stellar masses of $5\times10^{8-12}\,\mathrm{M}_{\odot}$ at $\mathrm{z=0}$ using the cosmological magnetohydrodynamical TNG50 simulation. Three different origins of stars comprise galactic centers: 1) in-situ (born in the center), 2) migrated (born elsewhere in the galaxy and ultimately moved to the center), 3) ex-situ (accreted from other galaxies). In-situ and migrated stars dominate the central stellar mass budget on average with 73% and 23% respectively. The ex-situ fraction rises above 1% for galaxies $\gtrsim10^{11}\,\mathrm{M}_{\odot}$. Yet, only 9% of all galaxies exhibit no ex-situ stars in their centers and the scatter of ex-situ mass is significant ($4-6\,\mathrm{dex}$). Migrated stars predominantly originate closely from the center ($1-2\,\mathrm{kpc}$), but if they travelled together in clumps distances reach $\sim10\,\mathrm{kpc}$. Central and satellite galaxies possess similar amounts and origins of central stars. Star forming galaxies ($\gtrsim10^{10}\,\mathrm{M}_{\odot}$) have on average more ex-situ mass in their centers than quenched ones. We predict readily observable stellar population and dynamical properties: 1) migrated stars are distinctly young ($\sim2\,\mathrm{Gyr}$) and rotationally supported, especially for Milky Way mass galaxies, 2) in-situ stars are most metal-rich and older than migrated stars, 3) ex-situ stars are on random motion dominated orbits and typically the oldest, most metal-poor and $\alpha$-enhanced population. We demonstrate that the interaction history with other galaxies leads to diverse pathways of building up galaxy centers in a $\Lambda$CDM universe. Our work highlights the necessity for cosmological context in formation scenarios of central galactic components and the potential to use galaxy centers as tracers of overall galaxy assembly. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.12890-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.12890) | **Euclid Preparation XXIX: Forecasts for 10 different higher-order weak  lensing statistics**  |
|| Euclid Collaboration, et al. -- incl., <mark>S. Heydenreich</mark>, <mark>L. Linke</mark> |
|*Appeared on*| *2023-01-31*|
|*Comments*| *33 pages, 24 figures, main results in Fig. 19 & Table 5, submitted to A&A*|
|**Abstract**| Recent cosmic shear studies have shown that higher-order statistics (HOS) developed by independent teams now outperform standard two-point estimators in terms of statistical precision thanks to their sensitivity to the non-Gaussian features of large-scale structure. The aim of the Higher-Order Weak Lensing Statistics (HOWLS) project is to assess, compare, and combine the constraining power of $10$ different HOS on a common set of $Euclid$-like mocks, derived from N-body simulations. In this first paper of the HOWLS series we compute the non-tomographic ($\Omega_{\rm m}$, $\sigma_8$) Fisher information for one-point probability distribution function, peak counts, Minkowski functionals, Betti numbers, persistent homology Betti numbers and heatmap, and scattering transform coefficients, and compare them to the shear and convergence two-point correlation functions in the absence of any systematic bias. We also include forecasts for three implementations of higher-order moments, but these cannot be robustly interpreted as the Gaussian likelihood assumption breaks down for these statistics. Taken individually, we find that each HOS outperforms the two-point statistics by a factor of around $2$ in the precision of the forecasts with some variations across statistics and cosmological parameters. When combining all the HOS, this increases to a $4.5$ times improvement, highlighting the immense potential of HOS for cosmic shear cosmological analyses with $Euclid$. The data used in this analysis are publicly released with the paper. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.12395-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.12395) | **J-PLUS: Towards an homogeneous photometric calibration using Gaia BP/RP  low-resolution spectra**  |
|| C. López-Sanjuan, et al. -- incl., <mark>H. Vázquez Ramió</mark>, <mark>A. Hernán-Caballero</mark>, <mark>C. Hernández-Monteagudo</mark> |
|*Appeared on*| *2023-01-31*|
|*Comments*| *Submitted to Astronomy and Astrophysics. 17 pages, 14 figures, 2 appendix. Comments are welcome. arXiv admin note: text overlap with arXiv:2101.12407*|
|**Abstract**| We present the photometric calibration of the twelve optical passbands for the Javalambre Photometric Local Universe Survey (J-PLUS) third data release (DR3), comprising 1642 pointings of two square degrees each. We selected nearly 1.5 million main sequence stars with a signal-to-noise ratio larger than ten in the twelve J-PLUS passbands and available low-resolution (R = 20-80) spectrum from the blue and red photometers (BP/RP) in Gaia DR3. We compared the synthetic photometry from BP/RP spectra with the J-PLUS instrumental magnitudes, after correcting for the magnitude and color terms between both systems, to obtain an homogeneous photometric solution for J-PLUS. To circumvent the current limitations in the absolute calibration of the BP/RP spectra, the absolute color scale was derived using the locus of 109 white dwarfs closer than 100 pc with a negligible interstellar extinction. Finally, the absolute flux scale was anchored to the Panoramic Survey Telescope and Rapid Response System (Pan-STARRS) photometry in the r band. The precision of the J-PLUS photometric calibration, estimated from duplicated objects observed in adjacent pointings and by comparison with the spectro-photometric standard star GD 153, is ~12 mmag in u, J0378, and J0395; and ~7 mmag in J0410, J0430, g, J0515, r, J0660, i, J0861, and z. The estimated accuracy in the calibration along the surveyed area is better than 1% for all the passbands. The Gaia BP/RP spectra provide a high-quality, homogeneous photometric reference in the optical range across the full-sky, in spite of their current limitations as an absolute reference. The calibration method for J-PLUS DR3 reaches an absolute precision and accuracy of 1% in the twelve optical filters within an area of 3284 square degrees. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.12919-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.12919) | **A strong X-ray polarization signal from the magnetar 1RXS  J170849.0-400910**  |
|| Silvia Zane, et al. -- incl., <mark>Jeremy Heyl</mark>, <mark>Herman L. Marshall</mark>, <mark>George G. Pavlov</mark>, <mark>Henric Krawczynski</mark> |
|*Appeared on*| *2023-01-31*|
|*Comments*| *14 pages, 5 Figures, accepted for publication on ApJL*|
|**Abstract**| Magnetars are the most strongly magnetized neutron stars, and one of the most promising targets for X-ray polarimetric measurements. We present here the first Imaging X-ray Polarimetry Explorer (IXPE) observation of the magnetar 1RXS J170849.0-400910, jointly analysed with a new Swift observation and archival NICER data. The total (energy and phase integrated) emission in the 2-8 keV energy range is linerarly polarized, at a ~35% level. The phase-averaged polarization signal shows a marked increase with energy, ranging from ~20% at 2-3 keV up to ~80% at 6-8 keV, while the polarization angle remain constant. This indicates that radiation is mostly polarized in a single direction. The spectrum is well reproduced by a combination of either two thermal (blackbody) components or a blackbody and a power law. Both the polarization degree and angle also show a variation with the spin phase, and the former is almost anti-correlated with the source counts in the 2-8 keV and 2-4 keV bands. We discuss the possible implications and interpretations, based on a joint analysis of the spectral, polarization and pulsation properties of the source. A scenario in which the surface temperature is not homogeneous, with a hotter cap covered by a gaseous atmosphere and a warmer region in a condensed state, provides a satisfactory description of both the phase- and energy-dependent spectro-polarimetric data. The (comparatively) small size of the two emitting regions, required to explain the observed pulsations, does not allow to reach a robust conclusion about the presence of vacuum birefringence effects. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.12976-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.12976) | **Resolved CO(1-0) emission and gas properties in luminous dusty star  forming galaxies at z=2-4**  |
|| F. Stanley, et al. -- incl., <mark>D. H. Hughes</mark> |
|*Appeared on*| *2023-01-31*|
|*Comments*| *25pages , 13 figures, Accepted for publication at ApJ*|
|**Abstract**| We present the results of a survey of CO(1-0) emission in 14 infrared luminous dusty star forming galaxies (DSFGs) at 2 < z < 4 with the NSF's Karl G. Jansky Very Large Array. All sources are detected in CO(1-0), with an ~1arcsec angular resolution. Seven sources show extended and complex structure. We measure CO luminosities of $({\mu})L'_{CO(1-0)}=0.4-2.9x10^{11}$ K km s$^{-1}$ pc$^2$, and molecular gas masses of (${\mu}$)M$_{H2}$ = 1.3 - 8.6 x 10$^{11}$ Mo, where ({\mu}) is the magnification factor. The derived molecular gas depletion times of t$_{\rm dep}$ = 40 - 460 Myr, cover the expected range of both normal star forming galaxies and starbursts. Comparing to the higher-J CO transitions previously observed for the same sources, we find CO temperature brightness ratios of r$_{32/10}$ = 0.4 - 1.4, r$_{43/10}$ = 0.4 - 1.7, and r$_{54/10}$ = 0.3 - 1.3. We find a wide range of CO spectral line energy distributions (SLEDs), in agreement with other high-z DSFGs, with the exception of three sources that are most comparable to the Cloverleaf and APM08279+5255. Based on radiative transfer modelling of the CO SLEDs we determine densities of n$_{H2}$ = 0.3 - 8.5 x 10$^3$ cm$^{-3}$ and temperatures of T$_K$ = 100 - 200 K. Lastly, four sources are detected in the continuum, three have radio emission consistent with their infrared derived star formation rates, while HerBS-70E requires an additional synchrotron radiation component from an active galactic nucleus. Overall, we find that even though the sample is similarly luminous in the infrared, by tracing the CO(1-0) emission a diversity of galaxy and excitation properties are revealed, demonstrating the importance of CO(1-0) observations in combination to higher-J transitions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.12872-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.12872) | **A Machine Learning approach for correcting radial velocities using  physical observables**  |
|| M. Perger, et al. -- incl., <mark>E. Herrero</mark> |
|*Appeared on*| *2023-01-31*|
|*Comments*| **|
|**Abstract**| Precision radial velocity (RV) measurements continue to be a key tool to detect and characterise extrasolar planets. While instrumental precision keeps improving, stellar activity remains a barrier to obtain reliable measurements below 1-2 m/s accuracy. Using simulations and real data, we investigate the capabilities of a Deep Neural Network approach to produce activity free Doppler measurements of stars. As case studies we use observations of two known stars (Eps Eridani and AUMicroscopii), both with clear signals of activity induced RV variability. Synthetic data using the starsim code are generated for the observables (inputs) and the resulting RV signal (labels), and used to train a Deep Neural Network algorithm. We identify an architecture consisting of convolutional and fully connected layers that is adequate to the task. The indices investigated are mean line-profile parameters (width, bisector, contrast) and multi-band photometry. We demonstrate that the RV-independent approach can drastically reduce spurious Doppler variability from known physical effects such as spots, rotation and convective blueshift. We identify the combinations of activity indices with most predictive power. When applied to real observations, we observe a good match of the correction with the observed variability, but we also find that the noise reduction is not as good as in the simulations, probably due to the lack of detail in the simulated physics. We demonstrate that a model-driven machine learning approach is sufficient to clean Doppler signals from activity induced variability for well known physical effects. There are dozens of known activity related observables whose inversion power remains unexplored indicating that the use of additional indicators, more complete models, and more observations with optimised sampling strategies can lead to significant improvements in our detrending capabilities. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure pic/aumic_QPC_b.png</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.11960-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.11960) | **A Global Inventory of Feedback**  |
|| <mark>Timothy M. Heckman</mark>, Philip N. Best |
|*Appeared on*| *2023-01-31*|
|*Comments*| **|
|**Abstract**| Feedback from both supermassive black holes and massive stars plays a fundamental role in the evolution of galaxies and the inter-galactic medium. In this paper we use available data to estimate the total amount of kinetic energy and momentum created per co-moving volume element over the history of the universe from three sources: massive stars and supernovae, radiation pressure and winds driven by supermassive black holes, and radio jets driven by supermassive black holes. Kinetic energy and momentum injection from jets peaks at z ~ 1, while the other two sources peak at z ~ 2. Massive stars are the dominant global source of momentum injection. For supermassive black holes, we find that the amount of kinetic energy from jets is about an order-of-magnitude larger than that from winds. We also find that amount of kinetic energy created by massive stars is about 2.5 epsilon times that carried by jets (where epsilon is the fraction of injected energy not lost to radiative cooling). We discuss the implications of these results for the evolution of galaxies and the IGM. Because the ratio of black hole mass to galaxy mass is a steeply increasing function of mass, we show that the relative importance of black hole feedback to stellar feedback likewise increases with mass. We show that there is a trend in the present-day universe which, in the simplest picture, is consistent with galaxies that have been dominated by black hole feedback being generally quenched, while galaxies that have been dominated by stellar feedback are star-forming. We also note that the amount of kinetic energy carried by jets and winds appears sufficient to explain the properties of hot gas in massive halos (> 10^13 solar masses). |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.12387-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.12387) | **Camera update for GONG refurbishment: Development and validation**  |
|| <mark>Anna L.H. Hughes</mark>, et al. |
|*Appeared on*| *2023-01-31*|
|*Comments*| *8 pages*|
|**Abstract**| This report provides a brief summary of the properties of new cameras selected for NSF's Global Oscillations Network Group (GONG) facilities operated by the NSO Integrated Synoptic Program (NISP). These camera replacements are part of a GONG refurbishment project aimed to extend GONG operations through roughly FY 2030. Testing has confirmed the suitability of the new cameras and that current data products would be largely unchanged. GONG magnetograms show approximately one-to-one scaling with old data, and the helioseismology data (l-nu diagrams) are nearly identical without any identifiable artifacts. A number of tests were also conducted for GONG processing pipelines and have demonstrated that the modified NISP data center pipelines can transition smoothly to processing observations taken with the new cameras |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.11942.md
    + _build/html/tmp_2301.11942/./SubhaloID_184937_fullhistory.png
    + _build/html/tmp_2301.11942/./SubhaloID_184937_fullhistory.png
    + _build/html/tmp_2301.11942/./SubhaloID_184937_fullhistory.png
exported in  _build/html/2301.12890.md
    + _build/html/tmp_2301.12890/./newplots/multifish2_proben2pcf_allprobes_allprobes_SLICS_f924_c128.png
    + _build/html/tmp_2301.12890/./plots/XipDerivativevsTheory_base.png
    + _build/html/tmp_2301.12890/./plots/XimDerivativevsTheory_base.png
    + _build/html/tmp_2301.12890/./plots/XiFisher_small_finite_AllScales.png
    + _build/html/tmp_2301.12890/./plots/XiFisher_small_finite_Valid.png
    + _build/html/tmp_2301.12890/./plots/fig_dustgrain_derivatives_small.png
    + _build/html/tmp_2301.12890/./plots/fisher_smallscale.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The Origin of Stars in the Inner 500 Parsecs in TNG50 Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.11942-b31b1b.svg)](https://arxiv.org/abs/2301.11942)<mark>Appeared on: 2023-01-31</mark> - _24 pages, 13 Figures, published in MNRAS_

</div>
<div id="authors">

Alina Boecker, et al. -- incl., <mark><mark>Nadine Neumayer</mark></mark>, <mark><mark>Annalisa Pillepich</mark></mark>, <mark><mark>Neige Frankel</mark></mark>, <mark><mark>Lars Hernquist</mark></mark>

</div>
<div id="abstract">

**Abstract:** \noindentWe investigate the origin of stars in the innermost 500 pc of galaxies spanning stellar masses of$5\times10^{8-12} \mathrm{M}_{\odot}$at$\mathrm{z=0}$using the cosmological magnetohydrodynamical TNG50 simulation. Three different origins of stars comprise galactic centers: 1) in-situ (born in the center), 2) migrated (born elsewhere in the galaxy and ultimately moved to the center), 3) ex-situ (accreted from other galaxies). In-situ and migrated stars dominate the central stellar mass budget on average with 73\%and 23\%respectively. The ex-situ fraction rises above 1\%for galaxies$\gtrsim10^{11} \mathrm{M}_{\odot}$. Yet, only 9\%of all galaxies exhibit no ex-situ stars in their centers and the scatter of ex-situ mass is significant ($4-6 \mathrm{dex}$). Migrated stars predominantly originate closely from the center ($1-2 \mathrm{kpc}$), but if they travelled together in clumps distances reach$\sim10 \mathrm{kpc}$. Central and satellite galaxies possess similar amounts and origins of central stars. Star forming galaxies ($\gtrsim10^{10} \mathrm{M}_{\odot}$) have on average more ex-situ mass in their centers than quenched ones. We predict readily observable stellar population and dynamical properties: 1) migrated stars are distinctly young ($\sim2 \mathrm{Gyr}$) and rotationally supported, especially for Milky Way mass galaxies, 2) in-situ stars are most metal-rich and older than migrated stars, 3) ex-situ stars are on random motion dominated orbits and typically the oldest, most metal-poor and$\alpha$-enhanced population. We demonstrate that the interaction history with other galaxies leads to diverse pathways of building up galaxy centers in a$\Lambda$CDM universe. Our work highlights the necessity for cosmological context in formation scenarios of central galactic components and the potential to use galaxy centers as tracers of overall galaxy assembly.

</div>

<div id="div_fig1">

<img src="tmp_2301.11942/./SubhaloID_184937_fullhistory.png" alt="Fig24" width="100%"/>

**Figure 24. -** \textbf{Central (500 pc) assembly history of an individual galaxy (\texttt{SubfindID} 184937) in TNG50 with a total stellar mass of}$\mathbf{10^{10.8} M_{\odot}}.$\textbf{This galaxy encompasses many mechanisms that can shape the stellar mass build-up in the center of galaxies in a}$\mathbf{\Lambda}$\textbf{CDM cosmology.}\textit{Top panel}: Points show all individual stellar particles that belong to that galaxy at $\mathrm{z}=0$. Their distance at the time of birth is shown with respect to their current host in the case of in-situ formed stars (\textit{light gray}: all in-situ particles, \textit{\textcolor{insitu}{pink}}: central in-situ stars only, \textit{\textcolor{migrated}{orange}}: central migrated stars only). In the case of the ex-situ formed stars the distance is shown with respect to their future host galaxy at the time of birth (\textit{color-coded according to the colorbar}: all ex-situ stars, \textit{\textcolor{exsitu}{blue}}: central ex-situ stars only). The distance to individual satellite galaxies (only with maximum stellar masses above $10^{6} \mathrm{M}_{\odot}$) that will merge with the primary at some point are shown with thinner solid lines. Their coloring also follows the colorbar, which visualizes the merger mass ratio taken at the time $\mathrm{t}_{\mathrm{max}}$, when the secondary galaxy reaches maximum stellar mass. The thick black solid line shows the radius of the FoF Group the galaxy belongs to at a given lookback time (represented as $\mathrm{R}_{200}$, where the group's density is 200 times the critical density of the Universe). The thick gray dashed line shows the distance between the individual galaxy and the central galaxy of the FoF group it belongs to. Approximately 7 Gyr ago the galaxy fell into another group and became a satellite galaxy. Before that it was the central of its own FoF group. The vertical black dotted line represents the time the kinetic AGN feedback starts to take effect, which quenches the center. This galaxy has 50\% in-situ, 30\% migrated (of which only 9\% are 'smoothly' migrated and the rest comes from migrated clumps) and 20\% ex-situ stars in its center. \textit{Bottom panel}: Histograms of formation times of \textcolor{insitu}{in-situ}(\textit{top}), formation (\textit{solid}) and arrival (\textit{dashed-dotted}) times at the center for \textcolor{migrated}{`smoothly' migrated}(\textit{middle}) as well as \textcolor{exsitu}{ex-situ} and \textcolor{migrated}{`clumpy' migrated}(\textit{bottom}) stars. Additionally, in the panel for the in-situ stars, we mark the time of coalescence for the six most massive mergers of this galaxy with thin blue colored solid lines. According to the colorbar of the top panel, a darker blue means a higher merger mass ratio. Pericenter passages for two mergers are shown by thin dashed lines following the same colorcode. The approximate time of the galaxy falling into its $\mathrm{z}=0$ FoF group is shown by the thick black solid line and the onset of the kinetic AGN feedback is shown as the black dotted line. In the panel for the `smoothly' migrated stars we also show the A$_2$ mode of the stars for a given lookback time (see Appendix \protect\ref{sec: galprops} for a definition). In the panel for the ex-situ and `clumpy' migrated stars, we show the time of coalescence of the two mergers that deposited ex-situ stars in the galaxy's center (\textit{blue solid lines}) as well as the three pericenter passages of the galaxy around its central galaxy after it became a satellite (\textit{gray dashed lines}).  (*fig: assembly*)

</div>
<div id="div_fig2">

<img src="tmp_2301.11942/./SubhaloID_184937_fullhistory.png" alt="Fig39" width="100%"/>

**Figure 39. -** \textbf{Central (500 pc) assembly history of an individual galaxy (\texttt{SubfindID} 184937) in TNG50 with a total stellar mass of}$\mathbf{10^{10.8} M_{\odot}}.$\textbf{This galaxy encompasses many mechanisms that can shape the stellar mass build-up in the center of galaxies in a}$\mathbf{\Lambda}$\textbf{CDM cosmology.}\textit{Top panel}: Points show all individual stellar particles that belong to that galaxy at $\mathrm{z}=0$. Their distance at the time of birth is shown with respect to their current host in the case of in-situ formed stars (\textit{light gray}: all in-situ particles, \textit{\textcolor{insitu}{pink}}: central in-situ stars only, \textit{\textcolor{migrated}{orange}}: central migrated stars only). In the case of the ex-situ formed stars the distance is shown with respect to their future host galaxy at the time of birth (\textit{color-coded according to the colorbar}: all ex-situ stars, \textit{\textcolor{exsitu}{blue}}: central ex-situ stars only). The distance to individual satellite galaxies (only with maximum stellar masses above $10^{6} \mathrm{M}_{\odot}$) that will merge with the primary at some point are shown with thinner solid lines. Their coloring also follows the colorbar, which visualizes the merger mass ratio taken at the time $\mathrm{t}_{\mathrm{max}}$, when the secondary galaxy reaches maximum stellar mass. The thick black solid line shows the radius of the FoF Group the galaxy belongs to at a given lookback time (represented as $\mathrm{R}_{200}$, where the group's density is 200 times the critical density of the Universe). The thick gray dashed line shows the distance between the individual galaxy and the central galaxy of the FoF group it belongs to. Approximately 7 Gyr ago the galaxy fell into another group and became a satellite galaxy. Before that it was the central of its own FoF group. The vertical black dotted line represents the time the kinetic AGN feedback starts to take effect, which quenches the center. This galaxy has 50\% in-situ, 30\% migrated (of which only 9\% are 'smoothly' migrated and the rest comes from migrated clumps) and 20\% ex-situ stars in its center. \textit{Bottom panel}: Histograms of formation times of \textcolor{insitu}{in-situ}(\textit{top}), formation (\textit{solid}) and arrival (\textit{dashed-dotted}) times at the center for \textcolor{migrated}{`smoothly' migrated}(\textit{middle}) as well as \textcolor{exsitu}{ex-situ} and \textcolor{migrated}{`clumpy' migrated}(\textit{bottom}) stars. Additionally, in the panel for the in-situ stars, we mark the time of coalescence for the six most massive mergers of this galaxy with thin blue colored solid lines. According to the colorbar of the top panel, a darker blue means a higher merger mass ratio. Pericenter passages for two mergers are shown by thin dashed lines following the same colorcode. The approximate time of the galaxy falling into its $\mathrm{z}=0$ FoF group is shown by the thick black solid line and the onset of the kinetic AGN feedback is shown as the black dotted line. In the panel for the `smoothly' migrated stars we also show the A$_2$ mode of the stars for a given lookback time (see Appendix \protect\ref{sec: galprops} for a definition). In the panel for the ex-situ and `clumpy' migrated stars, we show the time of coalescence of the two mergers that deposited ex-situ stars in the galaxy's center (\textit{blue solid lines}) as well as the three pericenter passages of the galaxy around its central galaxy after it became a satellite (\textit{gray dashed lines}).  (*fig: assembly*)

</div>
<div id="div_fig3">

<img src="tmp_2301.11942/./SubhaloID_184937_fullhistory.png" alt="Fig54" width="100%"/>

**Figure 54. -** \textbf{Central (500 pc) assembly history of an individual galaxy (\texttt{SubfindID} 184937) in TNG50 with a total stellar mass of}$\mathbf{10^{10.8} M_{\odot}}.$\textbf{This galaxy encompasses many mechanisms that can shape the stellar mass build-up in the center of galaxies in a}$\mathbf{\Lambda}$\textbf{CDM cosmology.}\textit{Top panel}: Points show all individual stellar particles that belong to that galaxy at $\mathrm{z}=0$. Their distance at the time of birth is shown with respect to their current host in the case of in-situ formed stars (\textit{light gray}: all in-situ particles, \textit{\textcolor{insitu}{pink}}: central in-situ stars only, \textit{\textcolor{migrated}{orange}}: central migrated stars only). In the case of the ex-situ formed stars the distance is shown with respect to their future host galaxy at the time of birth (\textit{color-coded according to the colorbar}: all ex-situ stars, \textit{\textcolor{exsitu}{blue}}: central ex-situ stars only). The distance to individual satellite galaxies (only with maximum stellar masses above $10^{6} \mathrm{M}_{\odot}$) that will merge with the primary at some point are shown with thinner solid lines. Their coloring also follows the colorbar, which visualizes the merger mass ratio taken at the time $\mathrm{t}_{\mathrm{max}}$, when the secondary galaxy reaches maximum stellar mass. The thick black solid line shows the radius of the FoF Group the galaxy belongs to at a given lookback time (represented as $\mathrm{R}_{200}$, where the group's density is 200 times the critical density of the Universe). The thick gray dashed line shows the distance between the individual galaxy and the central galaxy of the FoF group it belongs to. Approximately 7 Gyr ago the galaxy fell into another group and became a satellite galaxy. Before that it was the central of its own FoF group. The vertical black dotted line represents the time the kinetic AGN feedback starts to take effect, which quenches the center. This galaxy has 50\% in-situ, 30\% migrated (of which only 9\% are 'smoothly' migrated and the rest comes from migrated clumps) and 20\% ex-situ stars in its center. \textit{Bottom panel}: Histograms of formation times of \textcolor{insitu}{in-situ}(\textit{top}), formation (\textit{solid}) and arrival (\textit{dashed-dotted}) times at the center for \textcolor{migrated}{`smoothly' migrated}(\textit{middle}) as well as \textcolor{exsitu}{ex-situ} and \textcolor{migrated}{`clumpy' migrated}(\textit{bottom}) stars. Additionally, in the panel for the in-situ stars, we mark the time of coalescence for the six most massive mergers of this galaxy with thin blue colored solid lines. According to the colorbar of the top panel, a darker blue means a higher merger mass ratio. Pericenter passages for two mergers are shown by thin dashed lines following the same colorcode. The approximate time of the galaxy falling into its $\mathrm{z}=0$ FoF group is shown by the thick black solid line and the onset of the kinetic AGN feedback is shown as the black dotted line. In the panel for the `smoothly' migrated stars we also show the A$_2$ mode of the stars for a given lookback time (see Appendix \protect\ref{sec: galprops} for a definition). In the panel for the ex-situ and `clumpy' migrated stars, we show the time of coalescence of the two mergers that deposited ex-situ stars in the galaxy's center (\textit{blue solid lines}) as well as the three pericenter passages of the galaxy around its central galaxy after it became a satellite (\textit{gray dashed lines}).  (*fig: assembly*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\astroang}[1]{\ang[angle-symbol-over-decimal]{#1}}$
$\newcommand{\assign}[1]{\textcolor{red}{[#1]}}$
$\newcommand{\inprogress}[1]{\textcolor[rgb]{1,0.5,0}{[#1]}}$
$\newcommand{\done}[1]$
$\newcommand{\help}[1]{\textcolor{magenta}{[{\bf HELP!: #1}]}}$
$\newcommand{\nico}[1]{\textcolor{red}{[nico: #1]}}$
$\newcommand{\lucas}[1]{\textcolor{gray}{[lucas: #1]}}$
$\newcommand{\laila}[1]{\textcolor{magenta}{[laila: #1]}}$
$\newcommand{\cora}[1]{\textcolor{purple}{[cora: #1]}}$
$\newcommand{\vincenzo}[1]{\textcolor{cyan}{[vincenzo: #1]}}$
$\newcommand{\aoife}[1]{\textcolor{teal}{[aoife: #1]}}$
$\newcommand{\Sandrine}[1]{\textcolor{violet}{[Sandrine C.: #1]}}$
$\newcommand{\SP}[1]{\textcolor{teal}{[Sandrine P.: #1]}}$
$\newcommand{\ismael}[1]{\textcolor{brown}{[ismael: #1]}}$
$\newcommand{\sven}[1]{\textcolor{orange}{[sven: #1]}}$
$\newcommand{\simone}[1]{\textcolor{olive}{[Simone: #1]}}$
$\newcommand{\sihao}[1]{\textcolor{olive}{[sihao: #1]}}$
$\newcommand{\cgiocoli}[1]{\textbf{\textcolor{brilliantlavender}{[cgiocoli: #1]}}}$
$\newcommand{\MapMapMap}{\expval{\Map^3}}$
$\newcommand{\Mapn}{\expval{\Map^n}}$
$\newcommand{\varthetavec}{\vb*{\vartheta}}$
$\newcommand{\thetavec}{\vb*{\theta}}$
$\newcommand{\Map}{M_\mathrm{ap}}$
$\newcommand{\MapMap}{\expval{\Map^2}}$
$\newcommand{\ellvec}{\vb*{\ell}}$
$\newcommand{\orcid}[1]$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\astroang$}[1]{\ang[angle-symbol-over-decimal]{#1}}$
$\newcommand{$\assign$}[1]{\textcolor{red}{[#1]}}$
$\newcommand{$\inprogress$}[1]{\textcolor[rgb]{1,0.5,0}{[#1]}}$
$\newcommand{$\done$}[1]$
$\newcommand{$\help$}[1]{\textcolor{magenta}{[{\bf HELP!: #1}]}}$
$\newcommand{$\nico$}[1]{\textcolor{red}{[nico: #1]}}$
$\newcommand{$\lucas$}[1]{\textcolor{gray}{[lucas: #1]}}$
$\newcommand{$\laila$}[1]{\textcolor{magenta}{[laila: #1]}}$
$\newcommand{$\cora$}[1]{\textcolor{purple}{[cora: #1]}}$
$\newcommand{$\vincenzo$}[1]{\textcolor{cyan}{[vincenzo: #1]}}$
$\newcommand{$\aoife$}[1]{\textcolor{teal}{[aoife: #1]}}$
$\newcommand{$\Sandrine$}[1]{\textcolor{violet}{[Sandrine C.: #1]}}$
$\newcommand{$\SP$}[1]{\textcolor{teal}{[Sandrine P.: #1]}}$
$\newcommand{$\ismael$}[1]{\textcolor{brown}{[ismael: #1]}}$
$\newcommand{$\sven$}[1]{\textcolor{orange}{[sven: #1]}}$
$\newcommand{$\simone$}[1]{\textcolor{olive}{[Simone: #1]}}$
$\newcommand{$\sihao$}[1]{\textcolor{olive}{[sihao: #1]}}$
$\newcommand{$\cgiocoli$}[1]{\textbf{\textcolor{brilliantlavender}{[cgiocoli: #1]}}}$
$\newcommand{$\MapMapMap$}{\expval{$\Map$^3}}$
$\newcommand{$\Mapn$}{\expval{$\Map$^n}}$
$\newcommand{$\varthetavec$}{\vb*{\vartheta}}$
$\newcommand{$\thetavec$}{\vb*{\theta}}$
$\newcommand{$\Map$}{M_\mathrm{ap}}$
$\newcommand{$\Map$Map}{\expval{$\Map$^2}}$
$\newcommand{$\ellvec$}{\vb*{\ell}}$
$\newcommand{$\orcid$}[1]$</div>



<div id="title">

#  different higher-order weak lensing statistics

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.12890-b31b1b.svg)](https://arxiv.org/abs/2301.12890)<mark>Appeared on: 2023-01-31</mark> - _33 pages, 24 figures, main results in Fig. 19 & Table 5, submitted to A&A_

</div>
<div id="authors">

Euclid Collaboration, et al. -- incl., <mark><mark>S. Heydenreich</mark></mark>, <mark><mark>L. Linke</mark></mark>

</div>
<div id="abstract">

**Abstract:** Recent cosmic shear studies have shown that higher-order statistics (HOS) developed by independent teams now outperform standard two-point estimators in terms of statistical precision thanks to their sensitivity to the non-Gaussian features of large-scale structure. The aim of the Higher-Order Weak Lensing Statistics (HOWLS) project is to assess, compare, and combine the constraining power of$10$different HOS on a common set of\Euclid-like mocks, derived from N-body simulations. In this first paper of the HOWLS series we compute the non-tomographic ($\Omega_{\rm m}$,$\sigma_8$) Fisher information for one-point probability distribution function, peak counts, Minkowski functionals, Betti numbers, persistent homology Betti numbers and heatmap, and scattering transform coefficients, and compare them to the shear and convergence two-point correlation functions in the absence of any systematic bias. We also include forecasts for three implementations of higher-order moments, but these cannot be robustly interpreted as the Gaussian likelihood assumption breaks down for these statistics. Taken individually, we find that each HOS outperforms the two-point statistics by a factor of around$2$in the precision of the forecasts with some variations across statistics and cosmological parameters. When combining all the HOS, this increases to a$4.5$times improvement, highlighting the immense potential of HOS for cosmic shear cosmological analyses with\Euclid. The data used in this analysis are publicly released with the paper.

</div>

<div id="div_fig1">

<img src="tmp_2301.12890/./newplots/multifish2_proben2pcf_allprobes_allprobes_SLICS_f924_c128.png" alt="Fig10" width="100%"/>

**Figure 10. -** Individual Fisher forecasts in the $\sigma_8$-$\Omega_{\rm m}$ plane for a non-tomographic \Euclid-like survey for the $11$ statistics (orange) and $\gamma$-2PCF (blue), as well as their combination (green). The corresponding marginalized precision on CPs can be found in Table \ref{tab:forecasts}. The black dashed line in the first quadrant indicates constant $S_8=\sigma_8\sqrt{\Omega_{\rm m}/0.3}$. The bottom row with the grey shaded backgrounds displays probes that are not Gaussian distributed and cannot be robustly interpreted with Fisher forecasts. The abbreviated name of each summary statistic is displayed in the top-right part of each panel: $\gamma$-2PCF and $\kappa$-2PCF for the shear and convergence two-point correlation functions, $\kappa$-PDF for the convergence one-point probability distribution, peaks for aperture mass peak counts, MFs for convergence Minkowski functionals, BNs for convergence Betti numbers, pers. BNs and pers. heat. for aperture mass persistent homology Betti numbers and heatmap, ST for convergence scattering transform coefficients, HOM for higher-order convergence moments, and $\MapMapMap$ and $\Mapn$ for third and $n$-th order aperture mass moments. (*fig:Fishall*)

</div>
<div id="div_fig2">

<img src="tmp_2301.12890/./plots/XipDerivativevsTheory_base.png" alt="Fig9.1" width="25%"/><img src="tmp_2301.12890/./plots/XimDerivativevsTheory_base.png" alt="Fig9.2" width="25%"/><img src="tmp_2301.12890/./plots/XiFisher_small_finite_AllScales.png" alt="Fig9.3" width="25%"/><img src="tmp_2301.12890/./plots/XiFisher_small_finite_Valid.png" alt="Fig9.4" width="25%"/>

**Figure 9. -** \textit{Upper row:} A comparison of the derivatives of the $\gamma$-2PCF as predicted from theory (solid lines) and as measured from the DUSTGRAIN-\emph{pathfinder} simulations (dots). The errorbars are scaled to a single line-of-sight in the DUSTGRAIN-\emph{pathfinder} ensemble. The shaded regions display the scales that were discarded from the Fisher analysis presented in this work. In the lower panel we plot the relative deviation between the measured quantities and the theoretical predictions when accounting for (solid lines) or neglecting (dotted lines) finite field effects. The grey dashed lines display the 10 per cent errorband. \textit{Lower row:} A comparison of the Fisher forecast using the theoretical model (orange dashed) or the simulation measurements (blue dashed). The thin solid lines correspond to simulated analyses that are used to test the stability of the ellipses given the numerical noise in the derivatives. For the panel on the left we use all available scales on the $\gamma$-2PCF while for the figure on the right only the scales with realistic numerical derivatives (i.e. removing the grey shaded area of the upper panel) are included in the forecast. In the figure on the right we show for reference again the constraints of the theoretical model with all scales included. (*fig:biasnoise_gamma2pcf*)

</div>
<div id="div_fig3">

<img src="tmp_2301.12890/./plots/fig_dustgrain_derivatives_small.png" alt="Fig2.1" width="50%"/><img src="tmp_2301.12890/./plots/fisher_smallscale.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** \textit{Top:} The overall shape of the lensing $\kappa$-PDF, smoothed with a top-hat filter of radius \ang{;4.69;} and including shape noise. The fiducial cosmology is shown in grey as predicted from large deviation theory (line) and measured in the DUSTGRAIN-\emph{pathfinder} simulations (data points with error bars indicating the standard deviation across the 256 realisations). Derivatives of the $\kappa$-PDF with respect to the CPs are shown in colour, with theoretical predictions using non-linear variances predicted by Halofit shown as solid lines. The markers represent the derivatives obtained from the simulations using finite differences based on larger/smaller increments (points/stars). {\textit{Bottom:}} Fisher forecast constraints on $\Omega_{\rm m}$ and $\sigma_8$ for a \textit{Euclid}-like survey from the $\kappa$-PDF shown above. Numerically measured (orange) and theoretically predicted (pink) derivatives find good agreement, with differences in the final $1\sigma$ parameter constraints of approximately $20\%$. (*fig:PDF_dustgrain_theory*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

185  publications in the last 7 days.
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers